In [ ]:
from eose.instruments import  CircularGeometry, RectangularGeometry, BasicSensor

from pydantic import ValidationError

# Example usage of SphericalGeometry class used to define the field of view of the instrument

circ_geom = CircularGeometry(diameter=90.0)
print(circ_geom)

rect_geom = RectangularGeometry(angle_height= 90.0, angle_width= 45)
print(rect_geom)


In [ ]:
# Example simple usage of eose-api BasicSensor class

sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=30.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

print(sensor)

sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        orientation= list([0.5,0.5,0.5,0.5]),
                        field_of_view = RectangularGeometry(ngle_width=30.0, angle_height=10.0),
                        data_rate= 50.5,
                        bits_per_pixel= 8
                    )

print(sensor)

sensor = BasicSensor(   mass= 100.5,
                    volume= 0.75,
                    power= 150.0,
                    orientation= list([0.5,0.5,0.5,0.5]),
                    data_rate= 50.5,
                    bits_per_pixel= 8
                )

print(sensor)

display(sensor.model_dump_json())

In [3]:
%reset -f

## Example usage with TAT-C and InstruPy

In [1]:
from joblib import Parallel, delayed

import json
from datetime import datetime, timedelta, timezone
from shapely.geometry import box, mapping
from scipy.stats import hmean
import pandas as pd

from astropy.time import Time as AstroPy_Time

from pydantic import AwareDatetime

from tatc.schemas import Instrument as TATC_Instrument, Satellite as TATC_Satellite, TwoLineElements, Point
from tatc.analysis import collect_orbit_track, OrbitCoordinate, OrbitOutput
from tatc.analysis import (
    collect_multi_observations,
    aggregate_observations,
    reduce_observations,
)

from eose.propagation import (
    PropagationSample,
    PropagationRecord,
    PropagationRequest,
    PropagationResponse,
)
from eose.orbits import GeneralPerturbationsOrbitState, Propagator
from eose.satellites import Satellite
from eose.utils import CartesianReferenceFrame

from eose.access import (
    AccessSample,
    AccessRecord,
    AccessRequest,
    AccessResponse,
)
from eose.grids import UniformAngularGrid


from instrupy.basic_sensor_model import BasicSensorModel as InstruPy_BasicSensorModel
from instrupy import Instrument as InstruPy_Instrument

from eose.instruments import CircularGeometry, BasicSensor
from eose.datametrics import DataMetricsRequest, BasicSensorDataMetricsInstantaneous, DataMetricsSample, DataMetricsRecord, DataMetricsResponse


pd.set_option('display.max_rows', None)

### Define mission parameters

In [6]:
# define the orbit and the instrument
iss_omm_str = '[{"OBJECT_NAME":"ISS (ZARYA)","OBJECT_ID":"1998-067A","EPOCH":"2024-06-07T09:53:34.728000","MEAN_MOTION":15.50975122,"ECCENTRICITY":0.0005669,"INCLINATION":51.6419,"RA_OF_ASC_NODE":3.7199,"ARG_OF_PERICENTER":284.672,"MEAN_ANOMALY":139.0837,"EPHEMERIS_TYPE":0,"CLASSIFICATION_TYPE":"U","NORAD_CAT_ID":25544,"ELEMENT_SET_NO":999,"REV_AT_EPOCH":45703,"BSTAR":0.00033759,"MEAN_MOTION_DOT":0.00019541,"MEAN_MOTION_DDOT":0}]'
iss_omm = json.loads(iss_omm_str)[0]

basic_sensor = BasicSensor(   id="Atom",
                        mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=60.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

satellites=[
        Satellite(
            id="ISS",
            orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm),
            payloads=[
                basic_sensor
            ]
        )
    ]

targets = UniformAngularGrid(
        delta_latitude=20, delta_longitude=20, region=mapping(box(-180, -50, 180, 50))
    ).as_targets()

mission_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
mission_duration = timedelta(hours=2)
propagate_time_step = timedelta(minutes=0.5)

### Run propagation with TAT-C to get satellite states

In [ ]:
def propagate_tatc(request: PropagationRequest) -> PropagationResponse:
    if request.propagator != Propagator.SGP4:
        raise RuntimeError("TAT-C only supports SGP4 propagator.")
    orbit_tracks = Parallel(-1)(
        delayed(collect_orbit_track)(
            TATC_Satellite(
                name=satellite.id,
                orbit=TwoLineElements(tle=satellite.orbit.to_tle()),
            ),
            TATC_Instrument(name="Instrument"), # dummy entry
            pd.date_range(
                request.start,
                request.start + request.duration,
                freq=request.time_step,
            ),
            coordinates=(
                OrbitCoordinate.ECI
                if request.frame == CartesianReferenceFrame.ICRF
                else OrbitCoordinate.ECEF
            ),
            orbit_output=OrbitOutput.POSITION_VELOCITY,
        )
        for satellite in request.satellites
    )
    return PropagationResponse(
        **request.model_dump(exclude="satellite_records"),
        satellite_records=[
            PropagationRecord(
                satellite_id=satellite.id,
                samples=orbit_tracks[i].apply(
                    lambda r: PropagationSample(
                        time=r.time,
                        frame=request.frame,
                        position=r.geometry.coords[0],
                        velocity=r.velocity.coords[0],
                    ),
                    axis=1,
                ),
            )
            for i, satellite in enumerate(request.satellites)
        ],
    )



request = PropagationRequest(
    satellites=satellites,
    start=mission_start,
    duration=mission_duration,
    time_step=propagate_time_step,
    frame=CartesianReferenceFrame.ICRF,
    propagator=Propagator.SGP4
)

display(request.model_dump_json())

propagation_response = propagate_tatc(request)

#display(propagation_response.model_dump_json())

propagation_data = propagation_response.as_dataframe()

#display(propagation_data)


### Run access claculations with TAT-C to get access-periods at Target ground points. It is assumed that the sensor shall be of Circular FOV geometry. 
Coverage analysis is not required.

In [ ]:
def access_tatc(request: AccessRequest) -> AccessResponse:
    if request.propagator != Propagator.SGP4:
        raise RuntimeError("TAT-C only supports SGP4 propagator.")
    satellites = [
        TATC_Satellite(
            name=satellite.id,
            orbit=TwoLineElements(tle=satellite.orbit.to_tle()),
            instruments=[
                TATC_Instrument(name=str(payload.id), field_of_regard=payload.field_of_view.diameter) # assumed that the instrument has a circular FOV
            ],
        )
        for satellite in request.satellites
        for payload in satellite.payloads 
        if payload.id in request.payload_ids
    ]
    observations = Parallel(-1)(
        delayed(collect_multi_observations)(
            Point(
                id=i,
                longitude=target.position[0],
                latitude=target.position[1],
                altitude=(target.position[2] if len(target.position) > 2 else 0),
            ),
            satellites,
            request.start,
            request.start + request.duration,
        )
        for i, target in enumerate(request.targets)
    )
    return AccessResponse(
        **request.model_dump(exclude="target_records"),
        target_records=[
            (
                AccessRecord(target_id=target.id)
                if observations[i].empty
                else AccessRecord(
                    target_id=target.id,
                    samples=observations[i].apply(
                        lambda s: AccessSample(
                            start=s.start, duration=s.end - s.start, satellite_id=s.satellite, instrument_id=s.instrument
                        ),
                        axis=1,
                    ),
                )
            )
            for i, target in enumerate(request.targets)
        ],
    )

request = AccessRequest(
    satellites=satellites,
    targets=targets,
    start=mission_start,
    duration=mission_duration,
    propagator=Propagator.SGP4,
    payload_ids=["Atom"]
)

display(request.model_dump_json())

response = access_tatc(request)

#display(response.model_dump_json())

access_data = response.as_dataframe()

display(access_data)

### Run Data Metrics calculation with InstruPy

In [ ]:
def get_instantaneous_data_metrics_object(sensor_type: str, time_instant: AwareDatetime, data_metrics: dict) -> BasicSensorDataMetricsInstantaneous:
    """
    Create an instantaneous data metrics object for the specified sensor type.

    :param sensor_type: The type of the sensor (e.g., "BasicSensor").
    :paramtype sensor_type: str
    :param time_instant: The time at which the data metrics are recorded.
    :paramtype time_instant: AwareDatetime
    :param data_metrics: A dictionary containing data metrics such as incidence angle, look angle, etc.
    :paramtype data_metrics: dict
    :return: An instance of `BasicSensorDataMetricsInstantaneous` with the provided metrics.
    :rtype: BasicSensorDataMetricsInstantaneous
    :raises ValueError: If the sensor type is not supported.
    """
    if sensor_type == "BasicSensor":
        data_metrics_instantaneous = BasicSensorDataMetricsInstantaneous(
            time=time_instant,
            incidence_angle=data_metrics["incidence angle [deg]"],
            look_angle=data_metrics["look angle [deg]"],
            observation_range=data_metrics["observation range [km]"],
            solar_zenith=data_metrics["solar zenith [deg]"]
        )
        return data_metrics_instantaneous
    else:
        raise ValueError(f"{sensor_type} instrument type is not supported. Only 'BasicSensor' instrument type is supported.")


def data_metrics_instrupy(request: DataMetricsRequest) -> DataMetricsResponse:
    """
    Calculate data metrics using the provided request details and return the results.

    :param request: A `DataMetricsRequest` object containing details such as sensor specifications, start time, duration, etc.
    :paramtype request: DataMetricsRequest
    :return: A `DataMetricsResponse` object containing the calculated data metrics.
    :rtype: DataMetricsResponse
    :raises ValueError: If the sensor type is not supported.
    """
    sensor_type = request.sensor.type
    if sensor_type == "BasicSensor":
        basic_sensor = request.sensor
        # Define the InstruPy basic sensor model
        instrupy_sensor = InstruPy_BasicSensorModel.from_dict({
            "orientation": {"referenceFrame": "SC_BODY_FIXED", "convention": "REF_FRAME_ALIGNED"},
            "fieldOfViewGeometry": {"shape": "CIRCULAR", "diameter": basic_sensor.field_of_view.diameter}
        })
    else:
        raise ValueError(f"{sensor_type} instrument type is not supported. Only 'BasicSensor' instrument type is supported.")

    coverage_response = request.coverage_response
    prop_response = request.propagation_response
    dm_req_start = request.start
    dm_req_duration = request.duration

    def propagation_records_within_time_range(propagate_response, start_time, stop_time):
        """
        Return propagation records within the specified time range.

        :param propagate_response: The propagation response containing satellite states.
        :paramtype propagate_response: PropagationResponse
        :param start_time: The start time of the range.
        :paramtype start_time: AwareDatetime
        :param stop_time: The stop time of the range.
        :paramtype stop_time: AwareDatetime
        :return: A list of filtered propagation records within the time range.
        :rtype: list
        """
        filtered_propagate_records = []
        for propagate_record in propagate_response.records:
            if start_time <= propagate_record.time <= stop_time:
                filtered_propagate_records.append(propagate_record)
        return filtered_propagate_records

    dm_record = []  # Aggregation of data-metrics across multiple targets and multiple overpasses for each target.
    for cov_record in coverage_response.records:
        target = cov_record.target
        dm_sample = []  # Aggregation of data-metrics over multiple overpasses for the target
        for cov_sample in cov_record.samples:
            _start = max(dm_req_start, cov_sample.start)
            _stop = min(dm_req_start + dm_req_duration, cov_sample.start + cov_sample.duration)
            pr = propagation_records_within_time_range(prop_response, _start, _stop)

            dm_inst = []  # Aggregation of data-metrics over a single overpass (=coverage sample) for the target
            if pr:
                for _pr in pr:
                    time_utc = AstroPy_Time(_pr.time.astimezone(timezone.utc), scale='utc')
                    time_ut1 = time_utc.ut1
                    jd_ut1 = time_ut1.jd

                    SpacecraftOrbitState = {
                        'time [JDUT1]': jd_ut1,
                        'x [km]': _pr.position[0] * 1e-3,
                        'y [km]': _pr.position[1] * 1e-3,
                        'z [km]': _pr.position[2] * 1e-3,
                        'vx [km/s]': _pr.velocity[0] * 1e-3,
                        'vy [km/s]': _pr.velocity[1] * 1e-3,
                        'vz [km/s]': _pr.velocity[2] * 1e-3
                    }
                    TargetCoords = {
                        'lat [deg]': target.position[1],
                        'lon [deg]': target.position[0]
                    }
                    obsv_metrics = instrupy_sensor.calc_data_metrics(SpacecraftOrbitState, TargetCoords)

                    _dm_inst = get_instantaneous_data_metrics_object(sensor_type, _pr.time, obsv_metrics)
                    dm_inst.append(_dm_inst)

            _dm_sample = DataMetricsSample(instantaneous_metrics=dm_inst)
            dm_sample.append(_dm_sample)

        _dm_record = DataMetricsRecord(target=target, samples=dm_sample)
        dm_record.append(_dm_record)

    data_metrics_response = DataMetricsResponse(records=dm_record)
    return data_metrics_response


In [ ]:
request = DataMetricsRequest(
    sensor=basic_sensor,
    start=mission_start,
    duration=mission_duration,
    propagation_response=propagation_response,
    coverage_response=coverage_response
)

print("Data metrics request")
display(request.model_dump_json())

data_metrics_response = data_metrics_instrupy(request)

print("Data metrics response")
display(data_metrics_response.model_dump_json())



In [ ]:
display(data_metrics_response.records[4].model_dump_json())

In [ ]:
display(data_metrics_response.records[33].model_dump_json())